## Testing if mutation rate is elevated inside the CDR vs rest of the array

In [2]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import numpy as np

#### Pseudo code 
null_distr = empty list
for rep = 1..1000:
    SNV_rate_fake_CDRs = empty list 
    SNV_rate_outside_fake_CDRs = empty list
    for ref,query in sample pair:
       fake_CDR1 <- random location(s) the same size as the real CDR in ref sample
       fake_CDR2 <- random location(s) the same size as the real CDR in query sample
       SNV_rate_fake_CDRs+= SNV rate in fake CDR1, CDR2
       SNV_rate_outside_fake_CDRs += SNV rate outside fake CDR1, CDR2
    null_distr[rep] <- (mean(SNV_rate_fake_CDRs) - mean(SNV_rate_outside_fake_CDRs))

real_CDR_SNV_rates=empty list
outside_real_CDR_SNV_rates=empty list
for ref,query in sample pair:
    real_CDR_SNV_rates+= inside CDR SNV rates for query sample, ref sample
    outside_real_CDR_SNV_rates += outside CDR SNV rates for query sample, ref sample

test_stat <- (mean(real_CDR_SNV_rates) - mean(outside_real_CDR_SNV_rates))
p_value <- proportion of null_distr that is >test_stat

input data to read in:
- list of pairwise comparisons for each chrom (subset to pairs with dist < 0.2)
- SNV calls (bed format, 1 bed for each pair)
- CDR locations for each sample (also bed file)

#### Prepare data tables

- list of pairwise comparisons < 0.2 for each chrom 
- SNV raw mutation files for pairs < 0.2 
- CDR locations 

In [3]:
# Read in list of pairwise distances per chromosome

# Directory containing CSV files
pairwise_dist_dir = "/private/groups/patenlab/mira/centrolign/batch_submissions/centrolign/release2_QC_v2/all_pairs/distance_matrices/"  # this should be a folder path

# Collect all matching files
files = glob.glob(os.path.join(pairwise_dist_dir, "*_r2_QC_v2_centrolign_pairwise_distance.csv"))

all_dfs = []

for f in files:
    # Extract chr from filename
    # e.g. filename: "chr3_r2_QC_v2_centrolign_pairwise_distance.csv"
    basename = os.path.basename(f)
    chr_val = basename.split("_")[0]   # 'chr3'

    # Read CSV
    df = pd.read_csv(f, header=None,names=["sample1", "sample2", "direct_pairwise_dist","chr"])
    df["chr"] = chr_val
    all_dfs.append(df)

# Combine into one dataframe
all_pairs_dist_df = pd.concat(all_dfs, ignore_index=True)

all_pairs_dist_df=all_pairs_dist_df[all_pairs_dist_df["direct_pairwise_dist"]< 0.2]

all_pairs_dist_df["sample_pair"] = (
    all_pairs_dist_df[["sample1", "sample2"]]
    .apply(lambda x: "_".join(sorted(x)), axis=1)
)

all_pairs_dist_df.head()

,sample1,sample2,direct_pairwise_dist,chr,sample_pair
46,HG00741.2,HG01167.2,0.003675,chr6,HG00741.2_HG01167.2
984,NA19700.1,NA19835.1,0.079636,chr6,NA19700.1_NA19835.1
1224,HG02083.2,HG02129.2,0.176519,chr6,HG02083.2_HG02129.2
2003,HG02280.1,NA18879.1,0.134344,chr6,HG02280.1_NA18879.1
2046,HG02258.1,HG03583.2,0.105130,chr6,HG02258.1_HG03583.2


In [27]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path(
    "/private/groups/patenlab/mira/centrolign/analysis/SNVs_induced_pairwise_asm_coords"
)

# set for fast lookup
valid_pairs = set(all_pairs_dist_df["sample_pair"])

dfs = []

for chr_dir in BASE_DIR.iterdir():
    if not chr_dir.is_dir():
        continue
    
    
    chr_name = chr_dir.name
    print(f"Processing {chr_name}..")

    for bed_file in chr_dir.glob("*.bed"):
        # extract samples from filename
        s1, s2 = bed_file.stem.split("_", 1)
        pair_key = "_".join(sorted([s1, s2]))

        # skip if not in allowed pairs
        if pair_key not in valid_pairs:
            continue

        # read BED
        df = pd.read_csv(
            bed_file,
            sep="\t",
            header=None,
            names=["ref_contig", "ref_start", "ref_end","query_contig","query_start","query_end"]  # adjust if more BED columns exist
        )

        df["sample1"] = s1
        df["sample2"] = s2
        df["sample_pair"] = pair_key
        df["chr"] = chr_name

        dfs.append(df)

# combine all chromosomes
SNVs_raw_df = pd.concat(dfs, ignore_index=True)


Processing chr11..


KeyboardInterrupt: 

In [23]:
SNVs_raw_df.to_parquet("/private/groups/patenlab/mira/centrolign/analysis/CDR_permutation/SNVs_raw_df.parquet")

In [4]:
SNVs_raw_df=pd.read_parquet("/private/groups/patenlab/mira/centrolign/analysis/CDR_permutation/SNVs_raw_df.parquet")

In [24]:
SNVs_raw_df.head()

,ref_contig,ref_start,ref_end,query_contig,query_start,query_end,sample1,sample2,sample_pair,chr
0,HG00673#2#CM089162.1,50983877,50983878,HG02074#1#CM088274.1,50935171,50935172,HG00673.2,HG02074.1,HG00673.2_HG02074.1,chr11
1,HG00673#2#CM089162.1,50990315,50990316,HG02074#1#CM088274.1,50941609,50941610,HG00673.2,HG02074.1,HG00673.2_HG02074.1,chr11
2,HG00673#2#CM089162.1,50990316,50990317,HG02074#1#CM088274.1,50941610,50941611,HG00673.2,HG02074.1,HG00673.2_HG02074.1,chr11
3,HG00673#2#CM089162.1,50991378,50991379,HG02074#1#CM088274.1,50942672,50942673,HG00673.2,HG02074.1,HG00673.2_HG02074.1,chr11
4,HG00673#2#CM089162.1,50991379,50991380,HG02074#1#CM088274.1,50942673,50942674,HG00673.2,HG02074.1,HG00673.2_HG02074.1,chr11


In [5]:
import pandas as pd
from pathlib import Path

ASAT_DIR = Path(
    "/private/groups/patenlab/mira/centrolign/batch_submissions/centrolign/release2_QC_v2/per_smp_asat_beds"
)

dfs = []

for bed_file in ASAT_DIR.glob("*_asat_arrays.bed"):
    sample = bed_file.name.replace("_asat_arrays.bed", "")

    df = pd.read_csv(
        bed_file,
        sep="\t",
        header=None,
        names=["contig", "start", "end", "chr"]
    )

    df["sample"] = sample
    dfs.append(df)

asat_df = pd.concat(dfs, ignore_index=True)


In [9]:
asat_df.head()

,contig,start,end,chr,sample
0,HG02572#1#CM088151.1,92088720,94028031,chr2,HG02572.1
1,HG02572#1#CM088156.1,60815527,63871369,chr7,HG02572.1
2,HG02572#1#CM088157.1,44232206,46352903,chr8,HG02572.1
3,HG02572#1#CM088158.1,42616238,44955099,chr9,HG02572.1
4,HG02572#1#CM088160.1,50957725,52118575,chr11,HG02572.1


In [6]:
import pandas as pd
from pathlib import Path

BASE_DIR = Path(
    "/private/groups/migalab/jmmenend/HPRC/cenSatProject/CDR_data"
)
dfs = []

for bed_file in BASE_DIR.glob("*/ *.centrodip.bed".replace(" ", "")):
    # chr comes from directory name (and filename, they match)
    chr_name = bed_file.parent.name

    df = pd.read_csv(
        bed_file,
        sep="\t",
        header=None,
        usecols=[0, 1, 2],
        names=["contig", "start", "end"]
    )

    sample_parts = df["contig"].str.split("#", expand=True)
    df["sample"] = sample_parts[0] + "." + sample_parts[1]
    df["chr"] = chr_name
    dfs.append(df)

cdr_df = pd.concat(dfs, ignore_index=True)
cdr_df.head()

,contig,start,end,sample,chr
0,HG00097#1#CM094067.1,52461286,52582524,HG00097.1,chr11
1,HG00097#1#CM094067.1,52595575,52608024,HG00097.1,chr11
2,HG00097#2#CM094082.1,51960327,51997373,HG00097.2,chr11
3,HG00097#2#CM094082.1,52216289,52237877,HG00097.2,chr11
4,HG00097#2#CM094082.1,52242986,52270186,HG00097.2,chr11


### Helper functions 

#### Cigar parsing for SNV aligned bases 

In [7]:
import re

def parse_cigar(cigar):
    """
    Parse a CIGAR string into a list of (length, operation) tuples.

    Example:
        '10M2I5M3D8M' ->
        [(10,'M'), (2,'I'), (5,'M'), (3,'D'), (8,'M')]
    """
    return [
        (int(length), op)
        for length, op in re.findall(r'(\d+)([MIDNSHP=X])', cigar)
    ]

def reverse_cigar(parsed):
    """
    Convert a parsed CIGAR from ref->query into query->ref.

    Insertions become deletions and vice versa.
    Matches, mismatches, and equals stay the same.
    """
    converted = []
    for length, op in parsed:
        if op == "I":
            converted.append((length, "D"))
        elif op == "D":
            converted.append((length, "I"))
        else:
            converted.append((length, op))
    return converted

def build_aligned_intervals_from_ops(cigar_ops, start_pos):
    """
    Build aligned intervals on the genome from parsed CIGAR ops.

    Parameters
    ----------
    cigar_ops : list of (length, op)
        Parsed CIGAR string
    start_pos : int
        Genomic start position of the alignment

    Returns
    -------
    intervals : list of (start, end)
        Genomic intervals where bases are aligned (M, =, X)
    """
    intervals = []
    pos = start_pos

    for length, op in cigar_ops:
        if op in ("M", "=", "X"):
            # Aligned bases
            intervals.append((pos, pos + length))
            pos += length
        elif op in ("D", "N"):
            # Consumes reference but not query
            pos += length
        # Insertions do not consume reference positions

    return intervals

#### Computing overlaps

In [15]:
def aligned_bp(aligned_intervals, query_intervals):
    """
    Compute total aligned bases overlapping query intervals.

    Both aligned_intervals and query_intervals must be:
      - sorted
      - half-open intervals [start, end)

    Uses a two-pointer sweep (O(n + m)).

    Parameters
    ----------
    aligned_intervals : list of (start, end)
        Fixed alignment blocks from CIGAR
    query_intervals : list of (start, end)
        Arbitrary windows (real or fake CDRs, ASAT)

    Returns
    -------
    total_aligned_bp : int
    """
    total = 0
    i = j = 0

    while i < len(aligned_intervals) and j < len(query_intervals):
        a_start, a_end = aligned_intervals[i]
        q_start, q_end = query_intervals[j]

        overlap = min(a_end, q_end) - max(a_start, q_start)
        if overlap > 0:
            total += overlap

        # Advance the interval that ends first
        if a_end < q_end:
            i += 1
        else:
            j += 1

    return total

import numpy as np

def count_snvs_in_intervals(snv_positions, intervals):
    """
    Count SNVs falling inside a list of intervals.

    Parameters
    ----------
    snv_positions : 1D numpy array
        Sorted SNV positions for (sample, chr, sample_pair)
    intervals : list of (start, end)

    Returns
    -------
    n_snvs : int
    """
    total = 0
    for start, end in intervals:
        total += (
            np.searchsorted(snv_positions, end, side="left")
            - np.searchsorted(snv_positions, start, side="left")
        )
    return total

def intervals_overlap(a, b):
    return not (a[1] <= b[0] or a[0] >= b[1])


def random_intervals_like(real_intervals, bounds, rng, forbidden_intervals=None, max_tries=1000):
    """
    Sample non-overlapping fake intervals matching the lengths of real_intervals,
    within bounds, avoiding forbidden_intervals.
    """
    fake_intervals = []
    lengths = [end - start for start, end in real_intervals]

    if forbidden_intervals is None:
        forbidden_intervals = []

    for L in lengths:
        start_min = bounds[0]
        start_max = bounds[1] - L
        if start_max < start_min:
            raise ValueError("CDR length exceeds ASAT bounds")

        for _ in range(max_tries):
            s = rng.integers(start_min, start_max + 1)
            candidate = (s, s + L)

            # reject if overlaps real CDRs or previously placed fake CDRs
            if any(intervals_overlap(candidate, x)
                   for x in forbidden_intervals + fake_intervals):
                continue

            fake_intervals.append(candidate)
            break
        else:
            raise RuntimeError("Failed to place non-overlapping fake CDRs")

    return sorted(fake_intervals)


def snv_rate(
    snv_positions,
    aligned_intervals,
    query_intervals
):
    """
    Compute SNV rate = (# SNVs) / (aligned bases)
    for arbitrary query intervals.

    Returns NaN if aligned bases == 0.
    """
    aligned = aligned_bp(aligned_intervals, query_intervals)
    if aligned == 0:
        return np.nan

    n_snvs = count_snvs_in_intervals(snv_positions, query_intervals)
    return n_snvs / aligned

def complement_intervals(intervals, bounds):
    """
    Compute complement of intervals within given bounds.

    Parameters
    ----------
    intervals : list of (start, end)
        Intervals to exclude (e.g. CDRs)
    bounds : (start, end)
        ASAT bounds

    Returns
    -------
    outside_intervals : list of (start, end)
    """
    outside = []
    current = bounds[0]

    for start, end in sorted(intervals):
        if start > current:
            outside.append((current, start))
        current = max(current, end)

    if current < bounds[1]:
        outside.append((current, bounds[1]))

    return outside


### Pre-Processing data into dictionaries for faster lookup in permutation loop

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

# ---------------------------------------------
# 1. pairs_df
# ---------------------------------------------
pairs_df = all_pairs_dist_df[["sample1","sample2","sample_pair","chr"]].copy()

# ---------------------------------------------
# 2. real_cdr_intervals[(sample, chr)] -> list of (start,end)
# ---------------------------------------------
real_cdr_intervals = defaultdict(list)

for row in cdr_df.itertuples(index=False):
    contig, start, end, sample, chr_ = row
    key = (sample, chr_)
    real_cdr_intervals[key].append((start, end))

# Ensure intervals are sorted
for key in real_cdr_intervals:
    real_cdr_intervals[key].sort()

# ---------------------------------------------
# 4. asat_bounds[(sample, chr)] -> (start,end)
#    Only one row per sample in asat_df
# ---------------------------------------------
asat_bounds = {}
for row in asat_df.itertuples(index=False):
    contig, start, end, chr_, sample = row
    asat_bounds[(sample, chr_)] = (start, end)

# ---------------------------------------------
# 5. snv_pos[(sample, chr, sample_pair)] -> sorted np.array
# ---------------------------------------------
snv_pos = defaultdict(list)  # intermediate dict

# SNVs_raw_df has columns:
# ref_contig, ref_start, ref_end, query_contig, query_start, query_end, sample1, sample2, sample_pair, chr
for row in SNVs_raw_df.itertuples(index=False):
    # Ref sample SNVs
    key_ref = (row.sample1, row.chr, row.sample_pair)
    snv_pos[key_ref].append(row.ref_start)

    # Query sample SNVs
    key_query = (row.sample2, row.chr, row.sample_pair)
    snv_pos[key_query].append(row.query_start)

# Convert lists to sorted np.arrays
snv_pos = {k: np.sort(np.array(v)) for k,v in snv_pos.items()}


### Build aligned intervals from cigar strings for getting aligned bases in denominator  

In [11]:
# Build contig -> start mapping per sample
contig_start_map = {}
for row in asat_df.itertuples(index=False):
    contig_start_map[(row.sample, row.contig, row.chr)] = row.start

In [12]:
### Parse cigar strings to get aligned intervals per sample pair

import pandas as pd
import os

# assumes parse_cigar, reverse_cigar, build_aligned_intervals_from_ops are already defined

aligned_intervals = {}

# Folder containing CSVs per chromosome
csv_dir = "/private/groups/patenlab/mira/centrolign/analysis/variant_dist_CDR/aligned_bases_per_bed/dist_0.2_smp_contig_maps_cigars"

chroms = [f.replace(".contig_maps.csv","") 
          for f in os.listdir(csv_dir) if f.endswith(".contig_maps.csv")]

for chr_ in chroms:
    csv_path = os.path.join(csv_dir, f"{chr_}.contig_maps.csv")
    df = pd.read_csv(csv_path)

    for row in df.itertuples(index=False):
        s1, c1, s2, c2, cigar_path = row

        # skip missing files
        if not os.path.exists(cigar_path):
            print(f"Missing CIGAR file {cigar_path}, skipping {s1}-{s2}")
            continue

        # read the CIGAR string (assuming single line)
        with open(cigar_path) as f:
            cigar_string = f.readline().strip()

        # parse CIGAR
        parsed = parse_cigar(cigar_string)

        # build intervals
        ref_start = contig_start_map.get((s1, c1, chr_))
        query_start = contig_start_map.get((s2, c2, chr_))

        if ref_start is None or query_start is None:
            print(f"Missing start positions for {s1}-{c1} or {s2}-{c2}, skipping")
            continue

        ref_intervals = build_aligned_intervals_from_ops(parsed, ref_start)
        query_intervals = build_aligned_intervals_from_ops(reverse_cigar(parsed), query_start)

        # store in lookup dict
        pair_key = "_".join(sorted([s1, s2]))
        aligned_intervals[(s1, chr_, pair_key)] = ref_intervals
        aligned_intervals[(s2, chr_, pair_key)] = query_intervals



In [13]:
N = 4  # Number of lines to print

print(f"First {N} lines of the dictionary:")
for i, (key, value) in enumerate(aligned_intervals.items()):
    if i >= N:
        break
    print(f"{key}: {value}")

First 4 lines of the dictionary:
('HG01975.1', 'chr3', 'HG01975.1_HG02293.1'): [(91831800, 91845073), (91845073, 91845074), (91845074, 91845423), (91845423, 91845424), (91845424, 91870598), (91870598, 91870599), (91870599, 91877879), (91880432, 91881313), (91881313, 91881314), (91881314, 91883295), (91883295, 91883296), (91883296, 91889320), (91889320, 91889321), (91889321, 91891825), (91891825, 91891826), (91891826, 91892064), (91892064, 91892065), (91892065, 91902574), (91902574, 91902575), (91902575, 91906776), (91906776, 91906777), (91906777, 91911061), (91911061, 91911062), (91911062, 91915944), (91915944, 91915945), (91915945, 91916138), (91916138, 91916139), (91916139, 91921284), (91921284, 91921285), (91921285, 91921682), (91921682, 91921683), (91921683, 91928417), (91928417, 91928418), (91928418, 91928771), (91928771, 91932298), (91932298, 91934887), (91934887, 91934888), (91934888, 91934892), (91934892, 91934893), (91934893, 91944966), (91944966, 91944967), (91944967, 9194616

### Null distribution

In [34]:
import numpy as np

N_REPS = 1000
rng = np.random.default_rng(42)

null_distr = np.empty(N_REPS)

for rep in range(N_REPS):
    print(f"Rep: {rep}")

    fake_in_rates = []
    fake_out_rates = []

    for r in pairs_df.itertuples(index=False):
        s1, s2, pair, chr_ = r

        # skip pair if either sample has no real CDRs
        if (
            (s1, chr_) not in real_cdr_intervals
            or (s2, chr_) not in real_cdr_intervals
            or len(real_cdr_intervals[(s1, chr_)]) == 0
            or len(real_cdr_intervals[(s2, chr_)]) == 0
        ):
            continue

        for sample in (s1, s2):
            key = (sample, chr_)
            full_key = (sample, chr_, pair)

            if (
                key not in asat_bounds
                or full_key not in snv_pos
                or full_key not in aligned_intervals
            ):
                continue

            asat_start, asat_end = asat_bounds[key]
            real_cdrs = real_cdr_intervals[key]

            fake_cdr = random_intervals_like(
                real_intervals=real_cdrs,
                bounds=(asat_start, asat_end),
                rng=rng,
                forbidden_intervals=real_cdrs
            )

            fake_outside = complement_intervals(
                fake_cdr,
                (asat_start, asat_end)
            )

            pos = snv_pos[full_key]
            aln = aligned_intervals[full_key]

            rate_in = snv_rate(pos, aln, fake_cdr)
            rate_out = snv_rate(pos, aln, fake_outside)

            if np.isfinite(rate_in) and np.isfinite(rate_out):
                fake_in_rates.append(rate_in)
                fake_out_rates.append(rate_out)

    null_distr[rep] = np.mean(fake_in_rates) - np.mean(fake_out_rates)


Rep: 0


RuntimeError: Failed to place non-overlapping fake CDRs

In [26]:
print(null_distr)

[-3.50869634e-05  2.49409706e-05 -1.00355050e-04 -1.94175976e-05
 -4.01865704e-05  1.95472350e-05  4.08361342e-05 -6.60188624e-05
 -8.03277297e-05 -9.44621321e-05 -8.06985507e-05 -2.08220953e-05
  1.22172644e-04 -6.87713281e-05 -3.73363291e-05 -9.94702029e-05
 -2.82376947e-05 -6.77940646e-05 -3.61488663e-05 -1.28752978e-05
  2.80387878e-05  1.15098118e-04  2.61696047e-05 -3.10837067e-05
 -3.93285574e-05 -4.76137246e-05 -5.20059272e-05 -3.75896668e-06
  3.38423468e-05 -6.57875490e-05  1.21728969e-04 -1.01170806e-05
  3.00539220e-05 -2.29553341e-05  2.03955445e-05  7.84552586e-05
 -4.79040606e-05  1.77639022e-05  1.41459945e-04 -2.36637382e-05
 -2.37776265e-06  3.15899841e-05 -9.00689513e-06  1.84845908e-05
 -8.51752211e-05 -5.16758807e-05 -3.27747153e-05 -4.80349479e-05
  8.52002319e-05 -4.34078505e-05 -2.06611493e-05 -5.56963497e-05
 -4.89320768e-05 -7.65172090e-05  2.81194624e-05 -7.87922032e-05
 -8.75296458e-05  2.98186154e-05 -2.70121090e-05  1.48867209e-05
 -4.68143617e-05 -5.97755

### Test statistic 

In [ ]:
real_in_rates = []
real_out_rates = []

for r in pairs_df.itertuples(index=False):
    s1, s2, pair, chr_ = r
    print(s1, s2, pair, chr_)

    # skip entire pair if either sample has no real CDRs
    if (
        (s1, chr_) not in real_cdr_intervals
        or (s2, chr_) not in real_cdr_intervals
        or len(real_cdr_intervals[(s1, chr_)]) == 0
        or len(real_cdr_intervals[(s2, chr_)]) == 0
    ):
        continue

    for sample in (s1, s2):
        key = (sample, chr_)
        full_key = (sample, chr_, pair)

        if (
            key not in asat_bounds
            or full_key not in snv_pos
            or full_key not in aligned_intervals
        ):
            continue

        cdrs = real_cdr_intervals[key]
        asat_start, asat_end = asat_bounds[key]
        print(key, cdrs, asat_start, asat_end)

        outside = complement_intervals(
            cdrs,
            (asat_start, asat_end)
        )

        pos = snv_pos[full_key]
        aln = aligned_intervals[full_key]

        rate_in = snv_rate(
            snv_positions=pos,
            aligned_intervals=aln,
            query_intervals=cdrs
        )
        print(rate_in)

        rate_out = snv_rate(
            snv_positions=pos,
            aligned_intervals=aln,
            query_intervals=outside
        )
        print(rate_out)

        if np.isfinite(rate_in) and np.isfinite(rate_out):
            real_in_rates.append(rate_in)
            real_out_rates.append(rate_out)

    

HG03688.1 NA21106.1 HG03688.1_NA21106.1 chr12
('HG03688.1', 'chr12') [(34973545, 35028598), (35118261, 35154400), (35162236, 35187694)] 34801030 37739047
0.0010718766367434438
0.001150491727964476
('NA21106.1', 'chr12') [(35339221, 35393386), (35401527, 35447550), (35474360, 35531768), (35555700, 35565050), (35576946, 35594254)] 34786140 37669128
0.0008520422753030294
0.0011896808759210564
NA21102.2 NA21106.2 NA21102.2_NA21106.2 chr12
('NA21102.2', 'chr12') [(35019857, 35162028), (35245062, 35251848)] 34746794 37249151
0.0013843521543008244
0.0021152882105202066
('NA21106.2', 'chr12') [(34737897, 34749253), (34756565, 34784943), (34810920, 34819084), (34862737, 34951403), (34976688, 35009627)] 34632822 37056912
0.0033271137288402313
0.0020730939641188293
HG01258.1 HG02273.2 HG01258.1_HG02273.2 chr12
('HG01258.1', 'chr12') [(36716854, 36732299), (36746226, 36806019), (36816550, 36875665)] 34720867 38401101
0.0006053871795854247
0.0010992414692364008
('HG02273.2', 'chr12') [(37783657, 37

In [32]:
print(real_in_rates)

[np.float64(0.0010718766367434438), np.float64(0.0008520422753030294), np.float64(0.0013843521543008244), np.float64(0.0033271137288402313), np.float64(0.0006053871795854247), np.float64(0.0005599800895968143), np.float64(0.00036519931454897885), np.float64(0.00032270269067277945), np.float64(0.0015567765567765569), np.float64(0.001040088228173838), np.float64(0.0017285314610545825), np.float64(0.0016761937508058624), np.float64(0.0010530870143758623), np.float64(0.0008658222837918068), np.float64(0.003840849626870481), np.float64(0.002815481673299849), np.float64(0.0035917813255652814), np.float64(0.004071133810766298), np.float64(0.002351073249671766), np.float64(0.0029552224691474773), np.float64(0.001197540891278701), np.float64(0.0005385950363765377), np.float64(0.0018507272779850208), np.float64(0.0013979496738117428), np.float64(0.0005746280684788474), np.float64(0.0004205497426235575), np.float64(0.0017063212236760775), np.float64(0.0018297741763174942), np.float64(0.0163908211

In [33]:
test_stat = np.mean(real_in_rates) - np.mean(real_out_rates)
p_value = np.mean(null_distr > test_stat)
print(p_value)

0.003
